<a href="https://colab.research.google.com/github/mathluva/BERT-SQuAD/blob/main/SQuAD_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tf-models-official==2.4.0

     |████████████████████████████████| 1.1MB 3.5MB/s 
     |████████████████████████████████| 37.6MB 1.1MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 706kB 41.3MB/s 
     |████████████████████████████████| 174kB 34.1MB/s 
     |████████████████████████████████| 645kB 21.2MB/s 
     |████████████████████████████████| 358kB 43.7MB/s 
     |████████████████████████████████| 1.2MB 41.2MB/s 
     |████████████████████████████████| 102kB 7.0MB/s 


In [1]:
!pip install tf-models-nightly

     |████████████████████████████████| 1.6MB 6.9MB/s 
     |████████████████████████████████| 706kB 23.6MB/s 
     |████████████████████████████████| 1.2MB 30.3MB/s 
     |████████████████████████████████| 453.8MB 36kB/s 
     |████████████████████████████████| 37.6MB 1.1MB/s 
     |████████████████████████████████| 645kB 38.0MB/s 
     |████████████████████████████████| 174kB 46.5MB/s 
     |████████████████████████████████| 358kB 50.4MB/s 
     |████████████████████████████████| 61kB 6.6MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 4.4MB 48.0MB/s 
     |████████████████████████████████| 1.2MB 37.8MB/s 
     |████████████████████████████████| 471kB 45.9MB/s 
     |████████████████████████████████| 4.0MB 40.6MB/s 
     |████████████████████████████████| 6.0MB 44.8MB/s 
     |████████████████████████████████| 4.0MB 45.1MB/s 
     |████████████████████████████████| 3.9MB 27.8MB/s

In [2]:
!pip install sentencepiece

In [2]:
import tensorflow as tf
import tensorflow_hub as hub


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
from official import nlp
from official.nlp import bert


In [4]:
from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp.data.squad_lib import generate_tf_record_from_json_file

In [5]:
from official.nlp import optimization

from official.nlp.data.squad_lib import read_squad_examples
from official.nlp.data.squad_lib import FeatureWriter
from official.nlp.data.squad_lib import convert_examples_to_features
from official.nlp.data.squad_lib import write_predictions

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:43: UserWarning: You are currently using a nightly version of TensorFlow (2.6.0-dev20210402). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [6]:
import numpy as np
import math
import random
import time
import json
import collections
import os

from google.colab import drive


STAGE 2: DATA PREPROCESSING

In [7]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
input_meta_data = generate_tf_record_from_json_file(
        "/content/drive/MyDrive/train-v1.1.json",
        "/content/drive/MyDrive/Udemy/vocab.txt",
        "/content/drive/MyDrive/Udemy/train-v1.tf_record"
)

In [12]:
import io, json

In [16]:
with tf.io.gfile.GFile("/content/drive/MyDrive/Udemy/train_meta_data", "w") as writer:
    writer.write(json.dumps(input_meta_data, indent=4) + "\n")

In [17]:
BATCH_SIZE = 4

train_dataset = create_squad_dataset(
    "/content/drive/My Drive/Udemy/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # 384
    BATCH_SIZE,
    is_training=True)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


STAGE 3:  MODEL BUILDING


In [ ]:
#SQUAD LAYER
#two outputs, first list 
class BertSquadLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(BertSquadLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(
         units=2,
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)) #used by Google as optimal dense layer in finetuning BERT
                                                                                                                #truncated normal, get rid of extremely low and high values

  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2) 2 is number of output units

    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)] 
                                                                    #split logits, 2 *tensors of shape(batch_size, seq_len)
    return unstacked_logits[0], unstacked_logits[1]

In [ ]:
#WHOLE MODEL
class BERTSquad(tf.keras.Model):
    
    def __init__(self,
                 name="bert_squad"):
        super(BERTSquad, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)
        
        self.squad_layer = BertSquadLayer()
    
    def apply_bert(self, inputs):
        _ , sequence_output = self.bert_layer([inputs["input_word_ids"],
                                               inputs["input_mask"],
                                               inputs["input_type_ids"]])
        return sequence_output

    def call(self, inputs):
        seq_output = self.apply_bert(inputs)

        start_logits, end_logits = self.squad_layer(seq_output)
        
        return start_logits, end_logits